In [1]:
import pandas as pd
import numpy as np

In [2]:
#cargamos los datos diarios de precios de Ethereum, desde 01/01/2015 a 26/09/2021
eth_hist = pd.read_csv('./ETH-USD.csv')

In [3]:
eth_hist.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-08-07,2.831620,3.536610,2.521120,2.772120,2.772120,164329.0
1,2015-08-08,2.793760,2.798810,0.714725,0.753325,0.753325,674188.0
2,2015-08-09,0.706136,0.879810,0.629191,0.701897,0.701897,532170.0
3,2015-08-10,0.713989,0.729854,0.636546,0.708448,0.708448,405283.0
4,2015-08-11,0.708087,1.131410,0.663235,1.067860,1.067860,1463100.0
5,2015-08-12,1.058750,1.289940,0.883608,1.217440,1.217440,2150620.0
6,2015-08-13,1.222240,1.965070,1.171990,1.827670,1.827670,4068680.0
7,2015-08-14,1.810920,2.261880,1.754750,1.827870,1.827870,4637030.0
8,2015-08-15,1.802890,1.877240,1.570980,1.688900,1.688900,2554360.0
9,2015-08-16,1.684350,1.695240,1.089810,1.566030,1.566030,3550790.0


In [4]:
#Parece que tenemos algunos datos que faltan.

eth_hist[eth_hist.isna().any(axis=1)]

,Date,Open,High,Low,Close,Adj Close,Volume
1715,2020-04-17,NaN,NaN,NaN,NaN,NaN,NaN
1890,2020-10-09,NaN,NaN,NaN,NaN,NaN,NaN
1893,2020-10-12,NaN,NaN,NaN,NaN,NaN,NaN
1894,2020-10-13,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Añadimos a mano los datos, sacándolos de coinmarketcap.com
#creamos una copia con modificaciones para no tocar los datos originales

eth_hist_m=eth_hist.copy()

eth_hist_m.loc[[1894]]=['2020-10-13', 387.14, 387.38, 375.58,
        381.19, 381.19, 14226744838.0]
eth_hist_m.loc[[1893]]=['2020-10-12', 374.80, 395.12, 366.64,
        387.73, 387.73, 15867455424.0]
eth_hist_m.loc[[1890]]=['2020-10-09', 351.11, 368.31, 348.21,
        365.59, 365.59, 13043501763.0]
eth_hist_m.loc[[1715]]=['2020-04-17', 172.28, 174.28, 169.75,
        171.64, 171.64, 16714684266.0]

eth_hist_m[eth_hist_m.isna().any(axis=1)]

,Date,Open,High,Low,Close,Adj Close,Volume


In [6]:
#Transformamos Date a formato fecha
eth_hist_m['Date']=pd.to_datetime(arg=eth_hist_m['Date'],format='%Y-%m-%d')

In [7]:
#Quitamos las columnas que no necesitamos
eth_hist_m = eth_hist_m.drop(labels=['Open','High','Low','Adj Close'], axis=1)

In [8]:
eth_hist_m['dia']=eth_hist_m['Date'].dt.day_name()
eth_hist_m['mes']=eth_hist_m['Date'].dt.month

In [9]:
#añadimos columnas de variaciones de precio desde el día antes, semana antes y mes antes
eth_hist_m['C_dia_ant'] = eth_hist_m['Close'].shift(1)
eth_hist_m['C_dia_ant'].loc[[0]]=eth_hist_m['C_dia_ant'][1]

eth_hist_m['var_dia_ant']=100*eth_hist_m['Close']/eth_hist_m['C_dia_ant']-100

/home/dsc/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [10]:
eth_hist_m['C_sem_ant'] = eth_hist_m['Close'].shift(7)

for i in range(0,7):
    eth_hist_m['C_sem_ant'].loc[[i]]=eth_hist_m['C_sem_ant'][i+7]

eth_hist_m['var_sem_ant']=100*eth_hist_m['Close']/eth_hist_m['C_sem_ant']-100

In [11]:
eth_hist_m['C_mes_ant'] = eth_hist_m['Close'].shift(30)

for i in range(0,30):
    eth_hist_m['C_mes_ant'].loc[[i]]=eth_hist_m['C_mes_ant'][i+30]

eth_hist_m['var_mes_ant']=100*eth_hist_m['Close']/eth_hist_m['C_mes_ant']-100

In [12]:
#Parece que hay diferencias importantes entre los días de la semana (los miércoles sube mucho más que los lunes)
eth_hist_m.groupby('dia')['var_dia_ant'].mean()

dia
Friday       0.414953
Monday       0.404541
Saturday     0.457005
Sunday       0.515044
Thursday     0.538756
Tuesday      0.671332
Wednesday    0.705835
Name: var_dia_ant, dtype: float64

In [13]:
#ahora vamos a mirar si hay diferencia significativa en el número de veces que ha subido la cotización por día de la semana
#los "mejores" días cambian respecto a los de las magnitudes de las variaciones, pero sigue habiendo diferencias importantes
#en este caso hay un 7,5% más de posibilidades de que la cotización suba un sábado que de que suba un lunes
eth_hist_m.groupby('dia')['var_dia_ant'].apply(lambda x: (x>0).sum()/(x).count())*100

dia
Friday       52.024922
Monday       49.062500
Saturday     56.697819
Sunday       50.155763
Thursday     47.812500
Tuesday      49.687500
Wednesday    51.875000
Name: var_dia_ant, dtype: float64

In [14]:
#Parece que hay diferencias significativas entre los meses,
#pero no me atrevo a decir que esto sea representativo de nada
#porque tenemos muy pocos meses de cada (6 años y pico de datos)
eth_hist_m[(eth_hist_m['Date']<'2021-01-01') & (eth_hist_m['Date']>'2015-12-31')].groupby('mes')['var_mes_ant'].mean()

mes
1     42.838526
2     55.569954
3     59.113043
4     15.134984
5     53.303120
6     44.127869
7     -9.373597
8      8.314592
9     -6.046958
10    -0.601752
11     1.206236
12     7.636542
Name: var_mes_ant, dtype: float64

In [15]:
#añadimos columnas de variaciones de volumen desde el día antes
eth_hist_m['V_dia_ant'] = eth_hist_m['Volume'].shift(1)
eth_hist_m['V_dia_ant'].loc[[0]]=eth_hist_m['V_dia_ant'][1]

eth_hist_m['var_V_dia_ant']=100*eth_hist_m['Volume']/eth_hist_m['V_dia_ant']-100

/home/dsc/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [16]:
#ahora creamos el set de datos de verdad, que por ahora va a contener:
#los datos de variación de precios diarios de los últimos 30 días
#los datos de variación de volúmenes diarios de los últimos 30 días
#los datos de variación de precios semanales desde 4 semanas antes a 12 semanas antes
#los datos de variación de precios mensuales desde 3 meses antes a 11 meses antes
#el día de la semana
#si el precio subió (1) o no (0) al día siguiente. Esta es la columna objetivo
#todo esto desde el 1 de enero de 2017, más tarde separaremos los datos de training y de test
#(ethereum empezó a cotizar en agosto de 2015 y los primeros meses no parecen significativos)

columnasData=['varP0', 'varP1', 'varP2', 'varP3', 'varP4', 'varP5', 'varP6', 'varP7', 'varP8', 'varP9',
             'varP10', 'varP11', 'varP12', 'varP13', 'varP14', 'varP15', 'varP16', 'varP17', 'varP18', 'varP19',
             'varP20', 'varP21', 'varP22', 'varP23', 'varP24', 'varP25', 'varP26', 'varP27', 'varP28', 'varP29',
             'varPs30', 'varPs37', 'varPs44', 'varPs51', 'varPs58', 'varPs65', 'varPs72', 'varPs79', 'varPs86',
             'varPm93', 'varPm123', 'varPm153', 'varPm183', 'varPm213', 'varPm243', 'varPm273', 'varPm303',
             'varPm333',
             'varV0', 'varV1', 'varV2', 'varV3', 'varV4', 'varV5', 'varV6', 'varV7', 'varV8', 'varV9',
             'varV10', 'varV11', 'varV12', 'varV13', 'varV14', 'varV15', 'varV16', 'varV17', 'varV18', 'varV19',
             'varV20', 'varV21', 'varV22', 'varV23', 'varV24', 'varV25', 'varV26', 'varV27', 'varV28', 'varV29',
             'dia', 'subida']
data_for_use = pd.DataFrame(columns=columnasData)


for i in range(0,eth_hist_m.shape[0]-1):
    if eth_hist_m['Date'][i]>pd.to_datetime(arg='2016-12-31',format='%Y-%m-%d'):
        dtemp = pd.concat([pd.DataFrame(eth_hist_m['var_dia_ant'][i-29:i+1][::-1].values),
                            pd.DataFrame(eth_hist_m['var_sem_ant'][i-92:i-29][::-7].values),
                            pd.DataFrame(eth_hist_m['var_mes_ant'][i-362:i-92][::-30].values),
                            pd.DataFrame(eth_hist_m['var_V_dia_ant'][i-29:i+1][::-1].values),
                            pd.DataFrame([eth_hist_m['dia'][i]]),
                            pd.DataFrame([eth_hist_m['var_dia_ant'][i+1]>0])],
                          ignore_index=True, axis=0).T

        dtemp = dtemp.set_index(keys=[eth_hist_m['Date'].loc[[i]]])
        
        new_cols = {x: y for x, y in zip(dtemp.columns, data_for_use.columns)}

        dtemp = dtemp.rename(columns=new_cols)

        data_for_use = data_for_use.append(dtemp.rename(columns=new_cols))

In [17]:
data_for_use.to_csv('ethereum_data_for_use.csv')